In [ ]:
import os
import base64

import telethon
from telethon import TelegramClient, events, sync, utils
import vobject
import tqdm

In [ ]:
def create_vcard(first_name, last_name, phone, photo):
    """
    first_name: 
    last_name: 
    phone: 
    contact_image: contact image, should be bytes of a jpg image. E.g. as returned by telethon.client.download_profile_photo
    """
    
    if first_name is None: 
        first_name = ''
    if last_name is None: 
        last_name = ''
    
    vcard = vobject.vCard()

    vcard.add('n')
    vcard.n.value = vobject.vcard.Name(family=last_name, given=first_name)

    vcard.add('fn')
    vcard.fn.value = ("%s %s" % (first_name, last_name)).strip()

    vcard.add('tel')
    vcard.tel.type = ['CELL', 'VOICE', 'pref']
    vcard.tel.value = str(phone)

    if photo is not None:
        o = vcard.add('PHOTO;ENCODING=b;TYPE=image/jpeg')
        o.value = base64.b64encode(photo).decode('utf-8')

    return vcard

In [ ]:
print('You can find the Api Id and Hash here: https://my.telegram.org/apps')
api_id = input('Api Id: ')
api_hash = input('Api Has: ')

client = TelegramClient('my_session', api_id, api_hash)
await client.start()

me = await client.get_me()

In [ ]:
dialogs = await client.get_dialogs(limit=None)
len(dialogs)

In [ ]:
def is_user(entity):
    user_type = telethon.tl.types.User
    return isinstance(entity, user_type)

def has_phone(user): 
    return user.phone is not None
    
    
entities = [d.entity for d in dialogs]
users = [e for e in entities if is_user(e) and has_phone(e) and e.first_name != 'Telegram']

vcards = []
for user in tqdm.tqdm(users): 
    vcard = create_vcard(
        first_name = user.first_name,
        last_name = user.last_name,
        phone = user.phone,
        photo = await client.download_profile_photo(user, file=bytes, download_big=True)
    )
    vcards += [vcard]


In [ ]:
os.makedirs('exported_contacts', exist_ok=True)

for vcard in vcards:
    filename = 'exported_contacts/%s.vcf' % vcard.fn.value
    filename.replace(' ', '_')
    
    print('saving %s : %s' % (vcard.fn.value, vcard.tel.value))

    with open(filename, 'w') as f:
        f.write(vcard.serialize())